In [1]:
from skimage import io as skio
from skimage import filters
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage import morphology
import cv2

In [2]:
img_path = "C:\\Users\jason\\UT\Junior_1st_Semester\EE_460J\\finalProject\DataScienceFinalProject\Formals\\102170.216.jpg"
#img_path = 'http://i.stack.imgur.com/SYxmp.jpg'

img = skio.imread(img_path)
#sobel = filters.sobel(img)


light_spots = np.array((img > 245).nonzero()).T
#plt.plot(light_spots[:, 1], light_spots[:, 0], 'o')
#plt.imshow(img)
#plt.title('light spots in image')
dark_spots = np.array((img < 3).nonzero()).T
#dark_spots = np.array((img < 100).nonzero()).T
#plt.plot(dark_spots[:, 1], dark_spots[:, 0], 'o')
#plt.imshow(img)
#plt.title('dark spots in image')

In [39]:
bool_mask = np.zeros(img.shape, dtype=np.bool)
bool_mask[tuple(light_spots.T)] = True
bool_mask[tuple(dark_spots.T)] = True
seed_mask, num_seeds = ndi.label(bool_mask)
num_seeds

2

In [40]:
ws = morphology.watershed(blurred, seed_mask)
plt.imshow(ws)

NameError: name 'blurred' is not defined

In [2]:
from skimage import io as skio
from skimage import filters
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage import morphology
import cv2

In [7]:
#made with help from https://stackoverflow.com/questions/29313667/how-do-i-remove-the-background-from-this-kind-of-image
# replaces background with given color in bgr and saves to new path
    # default color = yellow bc yellow is not used as a shoe color
    #img_path = src, new_img_path = dest
def replace_background(img_path, new_img_path, color=(0.0,1.0,1.0)):
    #== Parameters =======================================================================
    BLUR = 21
    CANNY_THRESH_1 = 10
    CANNY_THRESH_2 = 200
    MASK_DILATE_ITER = 10
    MASK_ERODE_ITER = 10
    MASK_COLOR = color # yellow In BGR format
    #== Processing =======================================================================

    #-- Read image -----------------------------------------------------------------------
    img = cv2.imread(str(img_path))
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #-- Edge detection -------------------------------------------------------------------
    edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
    edges = cv2.dilate(edges, None)
    edges = cv2.erode(edges, None)
    #-- Find contours in edges, sort by area ---------------------------------------------
    contour_info = []
    _, contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # Previously, for a previous version of cv2, this line was: 
    #  contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # Thanks to notes from commenters, I've updated the code but left this note
    for c in contours:
        contour_info.append((
            c,
            cv2.isContourConvex(c),
            cv2.contourArea(c),
        ))
    contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
    max_contour = contour_info[0]
    #-- Create empty mask, draw filled polygon on it corresponding to largest contour ----
    # Mask is black, polygon is white
    mask = np.zeros(edges.shape)
    cv2.fillConvexPoly(mask, max_contour[0], (255))
    #-- Smooth mask, then blur it --------------------------------------------------------
    mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
    mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
    mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)
    mask_stack = np.dstack([mask]*3)    # Create 3-channel alpha mask
    #-- Blend masked img into MASK_COLOR background --------------------------------------
    mask_stack  = mask_stack.astype('float32') / 255.0          # Use float matrices, 
    img         = img.astype('float32') / 255.0                 #  for easy blending

    masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR) # Blend
    masked = (masked * 255).astype('uint8')                     # Convert back to 8-bit 

    #cv2.imshow('img', masked)                                   # Display
    #cv2.waitKey(5)
    cv2.imwrite(new_img_path, masked)

In [6]:
img_path = "C:\\Users\jason\\UT\Junior_1st_Semester\EE_460J\\finalProject\DataScienceFinalProject\Formals\\102170.216.jpg"
new_img_path = "C:\\Users\jason\\UT\Junior_1st_Semester\EE_460J\\finalProject\DataScienceFinalProject\Formals\\102170.216_new.jpg"
replace_background(img_path, new_img_path)